# Check Pensions
Payroll fraction was checked in check_payroll notebook. This section requires checking 5 methodologies: the statewide and each municipal. 

For the statewide we need to get total employer-side contributions and make sure that's correct. Then get pension payouts for each agency, make sure those are correct. Then make sure the calculations are correct

For each municipal PD we need to find correct pension line-item and make sure calculation is correct


## Set up environment

In [14]:
%load_ext autoreload
%autoreload 2

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/pensions/State_Pensions_2Ver.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

pensions statewide are 


/opt/anaconda3/lib/python3.7/importlib/__init__.py:169: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  _bootstrap._exec(spec, module)


,2015,2016,2017,2018,2019,2020
agency_class,,,,,,
Appeals_Court,NaN,2952414.962,3335181.424,3505983.980,3677631.972,NaN
CPCS,NaN,2744489.977,3095990.133,3467557.969,3893469.757,NaN
DAA,NaN,78403.102,78994.836,93746.009,110430.671,NaN
Parole_Board,NaN,5774263.196,6308207.127,6758994.796,7247381.040,NaN
State_Police,NaN,82641290.781,87645976.975,93983707.225,104510631.055,NaN
Suffolk_DA,NaN,1423446.254,1511664.035,1600690.820,1763099.500,NaN
Suffolk_Sheriff,NaN,10200452.842,11106626.290,12031941.134,13335456.190,NaN
Supreme_Judicial_Court,NaN,2176348.347,2306287.795,2188637.495,2270515.531,NaN
state_police,NaN,274354.777,173419.375,104277.998,144018.113,NaN


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np
from sodapy import Socrata

In [16]:
sys.path.insert(0, os.path.abspath("../.."))

In [17]:
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
client.timeout = 40

In [18]:
from Initialize_Agencies_3Ver import get_agencies
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes_2Ver.py:175: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()


percent criminal correction for trial court local pensions 2016   0.343
2017   0.335
2018   0.358
2019   0.380
dtype: float64


/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes_2Ver.py:169: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_final_costs()
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes_2Ver.py:175: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()


## Get statewide pension contributions

In [20]:
statewide_pension_contribution = pd.DataFrame(client.get("pegc-naaa",
                                                         where = "appropriation_name = '(06121020) STATE RETIREMENT BD COMMONWEALTH PENSION'", 
                                                         limit =999999))

In [21]:
statewide_pension_contribution["amount"] = statewide_pension_contribution["amount"].astype(float)
statewide_pension_contribution["budget_fiscal_year"] = statewide_pension_contribution["budget_fiscal_year"].astype(int)
statewide_contributions = statewide_pension_contribution.groupby("budget_fiscal_year").sum()["amount"].loc[yr]

In [22]:
statewide_contributions

budget_fiscal_year
2016   1695644301.330
2017   1775058311.520
2018   1825313115.460
2019   1909137852.920
Name: amount, dtype: float64

## Get statewide pension payouts

In [23]:
retirement_payouts = pd.DataFrame(client.get("pni4-392n",
                                             where = "Year>=2015",
                                             limit = 999999))

In [24]:
retirement_payouts["annual_amount"] = retirement_payouts["annual_amount"].astype(float)
retirement_payouts.rename(columns={"department_last_worked_in": "department"}, inplace=True)
retirement_payouts["department"].fillna("None",inplace=True)
retirement_payouts["year"] = retirement_payouts["year"].astype(int)
retirement_payouts =retirement_payouts[retirement_payouts["retirement_system"]=="MSERS"]

In [25]:
statewide_payouts_by_year = retirement_payouts.groupby("year").sum()["annual_amount"]

### Check Trial Court

In [26]:
pd.set_option("display.max_rows", 1000)
retirement_payouts[retirement_payouts["department"].str.contains("(?i)trial")].groupby("department").size().sort_values(ascending=False).head(200)

department
Trial Court Justice               1512
Industrial Accident/rehab.         479
Trial Court Administration         400
Industrial Accident/rehab.3        218
Trial Court Law Libraries          163
Industrial Accident Board           36
Hampden Court Ii % Trial Court      21
Adm.office/trial Court/sec.pro      18
Trial Court Cse Unit                12
dtype: int64

This is a lot of double checking to do lol. I think it's safe to assume I got nearly all. No one can say it's some serious mistake to not count 20 employees out of hundreds. 

Ok I just realized something: some of these former employees we know worked within suffolk county. What if we didn't apply the suffolk county correction for them? So develop a category called "local pensions." Ballpark how much money that is here

In [21]:
local_TRC_employers =  ["Suffolk Superior Court",
                                  "Chelsea District Court", "Suffolk Cty. Juvenile Court",
                                  "South Boston District Court", "South Boston District  Court",
                                  "East Boston District Court", "Boston Municipal Court"]
local_TRC_employees = retirement_payouts[retirement_payouts["department"].isin(local_TRC_employers)]
local_by_year = local_TRC_employees.groupby("year").sum()["annual_amount"]

In [22]:
(local_by_year/statewide_payouts_by_year)*statewide_contributions

2015           nan
2016   4472732.947
2017   5003860.742
2018   5077337.557
2019   5151116.936
2020           nan
dtype: float64

In [23]:
statewide_TRC = ["Trial Court Justice", "Probation",
                                                  "Admin.office/district Court", "Office Of Court Management",
                                                  "General Court", "Superior Court Probation",
                                                  "Trial Court Administration",
                                                  "Juvenile Court Administration"]
statewide_TRC_employees = retirement_payouts[retirement_payouts["department"].isin(statewide_TRC)]
statewide_TRC_by_year = statewide_TRC_employees.groupby("year").sum()["annual_amount"]

In [24]:
statewide_TRC

['Trial Court Justice',
 'Probation',
 'Admin.office/district Court',
 'Office Of Court Management',
 'General Court',
 'Superior Court Probation',
 'Trial Court Administration',
 'Juvenile Court Administration']

In [25]:
(statewide_TRC_by_year/statewide_payouts_by_year)*statewide_contributions

2015            nan
2016   75033857.192
2017   81237414.625
2018   85340899.544
2019   90972824.566
2020            nan
dtype: float64

Ok quickly eyeball math to make sure it's ok

In [11]:
TRC = agencies["trial_court"]

In [12]:
TRC.local_pensions

2016   4512975.250
2017   4843035.102
2018   5111432.890
2019   5230811.762
Name: trial_court_local, dtype: object

In [13]:
TRC.pensions

2016   72032012.742
2017   79892708.277
2018   84439139.121
2019   90116437.768
Name: trial_court_statewide, dtype: object

**Not checked yet!!**
Check again when I'm fresh.

## Check State Police

First question: are there state police former employers by a different name?

In [37]:
pd.set_option("display.max_rows",100)
retirement_payouts[(retirement_payouts["title_at_retirement"].str.contains("(?i)police"))&
                   (retirement_payouts["title_at_retirement"].str.contains("(?i)campus")==False)]\
.groupby(["department","title_at_retirement" ]).size().sort_values(ascending=False).head(100)

department                                title_at_retirement           
State Police                              State Police Trooper,1st Class    3157
                                          State Police Sergeant             1282
Environmental Affairs                     Environ Police Officer (a/b)       136
                                          Environ Police Off. (c) Sgt        100
                                          Environ Police Off (d) lt           93
State Police                              State Police Dispatcher I           86
                                          State Police Dispatcher Iii         43
                                          State Police Dispatcher ii          26
Fish & Wildlife & Recreat.veh.            Environmntl Police Officer I        18
University Of Mass/amherst                Univ Police Officer                 16
Fish & Wildlife & Recreat.veh.            Environ Police Off. (c) Sgt         14
State Police                        

In [29]:
mass_turnpike_cops = retirement_payouts[retirement_payouts["department"]=="Mass Turnpike Auth.st.police"]


,trans_no,last_name,first_name,date_of_retirement,title_at_retirement,department,monthly_amount,year,position_type,retirement_system,annual_amount
4571,S00408122,Beaton,Roderick,1992-07-09T00:00:00.000,N/A,Mass Turnpike Auth.st.police,3954.79,2015,Full,MSERS,47457.480
4826,S00408044,Barrett,Richard,1994-11-04T00:00:00.000,N/A,Mass Turnpike Auth.st.police,3394.02,2015,Full,MSERS,40728.240
4969,S00413543,Eiseman,Robert,2009-12-31T00:00:00.000,State Police Sergeant,Mass Turnpike Auth.st.police,6498.67,2015,Full,MSERS,77984.040
5003,S00415467,Godfrey,William,2005-12-30T00:00:00.000,N/A,Mass Turnpike Auth.st.police,5473.54,2015,Full,MSERS,65682.480
5060,S00426660,Stoddard,Larry,1990-09-17T00:00:00.000,N/A,Mass Turnpike Auth.st.police,4001.90,2015,Full,MSERS,48022.800
...,...,...,...,...,...,...,...,...,...,...,...
719691,S03278795,Noone,Richard,2008-06-28T00:00:00.000,N/A,Mass Turnpike Auth.st.police,7644.41,2020,Full,MSERS,91732.920
720462,S02531828,Roy,James,1995-05-31T00:00:00.000,N/A,Mass Turnpike Auth.st.police,4648.55,2020,Full,MSERS,55782.600
725097,S00419991,Lynch,Philip,2008-04-05T00:00:00.000,N/A,Mass Turnpike Auth.st.police,6736.22,2020,Full,MSERS,80834.640
727103,S00420810,Mason,Stoddard,1996-10-25T00:00:00.000,N/A,Mass Turnpike Auth.st.police,3640.05,2020,Full,MSERS,43680.600


## Check CPCS

## Check Suffolk DA

## Check Suffolk Sheriff

## Check DAA

## Check MA Sheriff's Association

## Check Parole Board

## Check CJT

## Check Supreme Judicial Court

## Check Appeals Court

## Check DOC (make sure it's wrong)

## Check Boston Pension Contributions

## Check Chelsea Pension Contributions

## Check Revere Pension Contributions

## Check Winthrop Pension Contributions